<font size=6  color=#0000FF> [LINMA 2472] - ALGORITHMS IN DATA SCIENCE <br><br> 
HW 2 (Part I): k-PCA for outlier detection!   </font> <br><br><br>

<font size=5  color=#0000FF>

<br> 

<font size=7 color=#009999> Homework 3 Privacy </font> <br>

<div style="text-align: right"> Version 1 (2021-10-27)</div>
GROUPE 10
<br><br>
</font>
<br>
<font size=5 color=#009999> Instruction </font> <br><br>
The goal of this third project is to work about data anonymization. We will work with a data set of 2000 records, with lots of personnal informations about patients. We need to design methods as seen in the lecture in order to protect the privacy of users in the dataset. We have completly freedom on the method to use, we just need to justify our choice. 

In [132]:
#Various import

import numpy as np
import pandas as pd
import datetime
import hashlib
import matplotlib.pyplot as plt
from collections import Counter
import math, statistics
import os

# get more than one output per Jupyter cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [133]:
# Import of the data set

df = pd.read_csv(
    "~/Documents/Studies/MA 1/Q1/Algorithms in Data Sciences/Project 3/dataset_HW3.csv")

df.head(5)


zip_path = "~/Documents/Studies/MA 1/Q1/Algorithms in Data Sciences/Project 3/matched_zips.csv"
df_zip = pd.read_csv(zip_path, dtype={"zip": 'string'})
zip_to_state = df_zip.set_index('zip')['state'].to_dict()

save_final_path = "~/Documents/Studies/MA 1/Q1/Algorithms in Data Sciences/Project 3/anonymized_dataset.csv"


,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,Amelia Morrow,female,12/1950,26904,High School,Retired,2,married,Africa,2,0.0,Own house,Alzheimer's disease
1,Lillian Cole,female,4/1940,78088,PhD/md,Retired,1,married,Africa,1,0.0,Rent flat,multiple sclerosis
2,Lillian Simpson,female,12/1966,51502,Bachelor,Employed,1,married,Asia,0,0.1,Rent flat,heart disease
3,Avery Richards,female,4/1943,54080,Bachelor,Retired,1,married,Africa,0,0.0,Rent room,endometriosis
4,Sophia Alvarado,female,10/1947,68785,High School,Retired,1,married,Europe,0,0.0,Rent flat,gastritis


Drop useless columns
---

In [134]:
df.nunique()

id                 1972
gender                2
dob                 752
zipcode            1983
education             5
employment            4
children              9
marital_status        4
ancestry              6
number_vehicles       4
commute_time         12
accommodation         5
disease              13
dtype: int64

Generate a unique id based on the hash of the initial id, the date of birth, the zipcode and a salt
---

In [135]:

df.id = df.apply(lambda x: hashlib.sha224((str(x.id) + str(x.dob) +
                 str(x.zipcode)).encode() + os.urandom(128)).hexdigest(), axis=1)





In [136]:
# Columns that we choose to remove

df = df.drop(columns=['ancestry','education','marital_status','number_vehicles'])


Change all zipcodes to corresponding state
---

In [137]:
# Replace the precise zipcode by a more general one

def find_state(zip_code):
    """
        Return the state corresponding to the zipcode
    """
    return zip_to_state.get(str(zip_code)[0:3], "Unknown")


df['zipcode'] = df.zipcode.apply(find_state)
df.rename(columns={'zipcode': 'state'}, inplace=True)
df.head(5)


,id,gender,dob,state,employment,children,commute_time,accommodation,disease
0,9b458418f849bb3e3b4fddac0f74488a568abba21aa0de...,female,12/1950,Unknown,Retired,2,0.0,Own house,Alzheimer's disease
1,4b91f8e668c232635e7637e3ce21b1acf04291cddf3f77...,female,4/1940,(TX) Texas,Retired,1,0.0,Rent flat,multiple sclerosis
2,cb079ec3ffa5cce8c8b9163eff4f8ca15b290be4ad88d7...,female,12/1966,(IA) Iowa,Employed,1,0.1,Rent flat,heart disease
3,9bf1d2f6590d596ef1ac918c720d28db22632b9ecd78f7...,female,4/1943,(WI) Wisconsin,Retired,1,0.0,Rent room,endometriosis
4,4833d00b5f560752215b61c2115a4835c34afc1354e6a5...,female,10/1947,(NE) Nebraska,Retired,1,0.0,Rent flat,gastritis


Generalize semi-qualifier columns
---

In [138]:
def has_child(child):
    if child == 0:
        return False
    return True


def change_accom(accomod):
    _ , loge = accomod.split(' ')
    return loge


def change_education(educ):
    if educ == "PhD/md" or educ == "Masters":
        return "Masters+"
    return educ


def change_dob(dob):
    _ , year = dob.split('/')
    year = int(year)
    tens = 10*(year//10)
    return str(tens) + " - " + str(tens+10)
    

def more_than_1_hour(x):
    if isinstance(x, str):
        return x
    elif x == 0.0:
        return '[0]'
    elif x <= 1.0:
        return '[0-1]'
    elif x > 1.0:
        return '[1+]'

def vehicles(x):
    int(x)
    if x > 0:
        return True
    else: 
        return False
    

q3 = df.commute_time.quantile(q=0.75)

df.dob = df.dob.apply(change_dob)

df.accommodation = df.accommodation.apply(change_accom)

# df.education = df.education.apply(change_education)

# df.number_vehicles = df.number_vehicles.apply(vehicles)

# df.rename(columns={'number_vehicles': 'vehicles'}, inplace=True)

df.children = df.children.apply(has_child)

df.commute_time = df.commute_time.apply(more_than_1_hour)

df.head(5)



,id,gender,dob,state,employment,children,commute_time,accommodation,disease
0,9b458418f849bb3e3b4fddac0f74488a568abba21aa0de...,female,1950 - 1960,Unknown,Retired,True,[0],house,Alzheimer's disease
1,4b91f8e668c232635e7637e3ce21b1acf04291cddf3f77...,female,1940 - 1950,(TX) Texas,Retired,True,[0],flat,multiple sclerosis
2,cb079ec3ffa5cce8c8b9163eff4f8ca15b290be4ad88d7...,female,1960 - 1970,(IA) Iowa,Employed,True,[0-1],flat,heart disease
3,9bf1d2f6590d596ef1ac918c720d28db22632b9ecd78f7...,female,1940 - 1950,(WI) Wisconsin,Retired,True,[0],room,endometriosis
4,4833d00b5f560752215b61c2115a4835c34afc1354e6a5...,female,1940 - 1950,(NE) Nebraska,Retired,True,[0],flat,gastritis


Compute the entropy for all possible columns combination in order to find the best entropy
---

In [139]:
def entropy(C, N=2000): 
    return round(- sum([(ci/N) * math.log(ci/N, 2) for i, ci in enumerate(C)]), 4)

# 8.3858




In [140]:
from itertools import *
from tqdm import tqdm


covariates = list(df)
covariates = [e for e in covariates if e not in ('id', 'disease')]
results = []

for c in tqdm(range(1, len(covariates)+1)):
    for used in combinations(covariates, c):
        used = used + ("disease",)
        used = list(used)
        ct = df.groupby(used).size().to_frame("count")
        len_groups = len(ct)
        len_unique = sum(ct['count'] < 2)

        C = ct['count'].to_numpy()
        CR = C[C > 1]

        size_cr = 2000-len_unique

        entropy_full = entropy(C)
        entropy_cut = entropy(CR, size_cr)

        results.append(
            {
                "used": used,
                "len_groups": len_groups,
                "len_unique": len_unique,
                "diff": size_cr,
                "entropy_full": entropy_full,
                "entropy_cut": entropy_cut
            }
        )


100%|██████████| 7/7 [00:00<00:00, 25.95it/s]


In [141]:
results_sort = sorted(results, key=lambda x: (
    x['diff'], x['len_groups'],  x['entropy_cut'], x['entropy_full'], ), reverse=True)
results_filter = [dic for dic in results_sort if dic.get(
    'len_unique') > 0 and dic.get('len_unique') < 100 and dic.get('entropy_full') > 5]
results_filter_sorted = sorted(
    results_filter, key=lambda x: x['entropy_cut'], reverse=True)

results_filter_sorted[:5]




[{'used': ['gender', 'dob', 'accommodation', 'disease'],
  'len_groups': 428,
  'len_unique': 89,
  'diff': 1911,
  'entropy_full': 8.3858,
  'entropy_cut': 8.1999},
 {'used': ['gender', 'dob', 'commute_time', 'disease'],
  'len_groups': 419,
  'len_unique': 81,
  'diff': 1919,
  'entropy_full': 8.2499,
  'entropy_cut': 8.0757},
 {'used': ['gender', 'employment', 'commute_time', 'accommodation', 'disease'],
  'len_groups': 332,
  'len_unique': 72,
  'diff': 1928,
  'entropy_full': 7.772,
  'entropy_cut': 7.5998},
 {'used': ['dob', 'accommodation', 'disease'],
  'len_groups': 256,
  'len_unique': 42,
  'diff': 1958,
  'entropy_full': 7.5545,
  'entropy_cut': 7.4507},
 {'used': ['gender', 'dob', 'children', 'disease'],
  'len_groups': 275,
  'len_unique': 72,
  'diff': 1928,
  'entropy_full': 7.5852,
  'entropy_cut': 7.406}]

Drop all unique quasi-identifiers and those who do not respect the 2-diversity
---

In [142]:
quasi_identifiers = list(df)
quasi_identifiers = [e for e in quasi_identifiers if e not in ('id', 'disease','state')]
sensitive_column = "disease"

gbqi = df[quasi_identifiers+[sensitive_column]].groupby(quasi_identifiers)
indexes_to_drop = gbqi.filter(
    lambda x: x[sensitive_column].nunique() == 1).index
final_df = df[["id"]+quasi_identifiers+[sensitive_column]].drop(indexes_to_drop)


In [143]:
if input(f"Save the dataset to {save_final_path} ? [y/n]").lower() in ["y", "yes"]:
    df.to_csv(save_final_path, index=False)
    print(f"CSV saved to {save_final_path}")

print(f"Percentage dropped to respect 2-anonymity and 2-diversity = {100*len(indexes_to_drop)/len(df)} %")
final_df.head(5)

CSV saved to ~/Documents/Studies/MA 1/Q1/Algorithms in Data Sciences/Project 3/anonymized_dataset.csv
Percentage dropped to respect 2-anonymity and 2-diversity = 4.3 %


,id,gender,dob,employment,children,commute_time,accommodation,disease
0,9b458418f849bb3e3b4fddac0f74488a568abba21aa0de...,female,1950 - 1960,Retired,True,[0],house,Alzheimer's disease
1,4b91f8e668c232635e7637e3ce21b1acf04291cddf3f77...,female,1940 - 1950,Retired,True,[0],flat,multiple sclerosis
2,cb079ec3ffa5cce8c8b9163eff4f8ca15b290be4ad88d7...,female,1960 - 1970,Employed,True,[0-1],flat,heart disease
3,9bf1d2f6590d596ef1ac918c720d28db22632b9ecd78f7...,female,1940 - 1950,Retired,True,[0],room,endometriosis
4,4833d00b5f560752215b61c2115a4835c34afc1354e6a5...,female,1940 - 1950,Retired,True,[0],flat,gastritis


In [144]:
def is_l_diverse(df: pd.DataFrame, sensitive_column: str, feature_columns: list, l: int = 2) -> bool:
    """determine if the dataset respects the l-diversity
       source : https://en.wikipedia.org/wiki/L-diversity

    Args:
        df (pd.DataFrame): the dataset
        sensitive_column (str): the sensitive column to preserve from anonymity
        feature_columns (list): the feature/quasi-identifiers columns ot look for the l-diversity
        l (int, optional): The corresponding l to respect. Defaults to 2.

    Returns:
        bool: True if the number of unique sensitive informations of all set of quasi-identifiers are greater or equal to l
    """
    gbdc = df[[sensitive_column] +
              feature_columns].groupby(feature_columns)  # Group by feature columns
    _is_l_diverse = (gbdc[sensitive_column].nunique() >= l).all()
    return _is_l_diverse


def is_k_anonym(df: pd.DataFrame, feature_columns: str, k: int = 2) -> bool:
    """Determine if the dataset respect the k-anonymity
       source : https://en.wikipedia.org/wiki/K-anonymity

    Args:
        df (pd.DataFrame): the dataset
        feature_columns (list): the feature/quasi-identifiers columns ot look for the k-anonymity
        k (int, optional): The corresponding k to respect. Defaults to 2.

    Returns:
        [bool]: True if the count of all set of quasi-identifiers are greater or equal to k
    """

    gbdc = df[feature_columns].groupby(
        feature_columns)  # Group by feature columns
    _is_k_anonym = (gbdc[feature_columns[0]].count().values >= k).all()
    return _is_k_anonym

k, l = 2, 2
_is_k_anonym = is_k_anonym(final_df, quasi_identifiers, k)
_is_l_diverse = is_l_diverse(final_df, sensitive_column, quasi_identifiers, l)

print(f"Is the final dataset {k}-anonym? {_is_k_anonym}")
print(f"Is the final dataset {l}-diverse? {_is_l_diverse}")


Is the final dataset 2-anonym? True
Is the final dataset 2-diverse? True


In [145]:
def get_sensitive_frequences(df: pd.DataFrame, sensitive_column: str) -> dict:
    """get the frequencies for each sensitive value

    Returns:
        dict: a dictionnary with the coreesponding frequency for each sensitive value
    """
    total_count = len(df)
    sensitive_freqs = dict()
    counts = df.groupby(sensitive_column)[sensitive_column].agg('count')
    for value, count in counts.to_dict().items():
        sensitive_freqs[value] = count / total_count
    return sensitive_freqs


def t_closeness(df: pd.DataFrame, sensitive_column: str, feature_columns: list) -> int:
    """return the maximum distance between the distribution of a sensitive value in a \
    set of quasi-identifiers and the distribution of this same sensitive value in the \
    all database

    Args:
        df (pd.DataFrame): the dataset
        sensitive_column (str): the sensitive column to preserve from anonymity
        feature_columns (list): the feature/quasi-identifiers columns ot look for the l-diversity

    Returns:
        int: the maximum distance
    """
    # First get all frequences for each unique sensitive value
    sensitive_freqs = get_sensitive_frequences(df, sensitive_column)
    d_max = 0.
    gb = df[[sensitive_column] +
            feature_columns].groupby(feature_columns)[sensitive_column]

    for idx, (name, group) in enumerate(gb):
        sensitive_values = group.values
        for value in group.unique():
            p = sum(sensitive_values == value) / len(group)
            distance = abs(sensitive_freqs[value] - p)
            d_max = max(d_max, distance)
    return d_max


t_final = t_closeness(final_df, sensitive_column,quasi_identifiers)
print(t_final)


0.5966562173458725


In [146]:

t_init = t_closeness(df, sensitive_column, quasi_identifiers)
print(t_init)


0.965


In [147]:
final_df

,id,gender,dob,employment,children,commute_time,accommodation,disease
0,9b458418f849bb3e3b4fddac0f74488a568abba21aa0de...,female,1950 - 1960,Retired,True,[0],house,Alzheimer's disease
1,4b91f8e668c232635e7637e3ce21b1acf04291cddf3f77...,female,1940 - 1950,Retired,True,[0],flat,multiple sclerosis
2,cb079ec3ffa5cce8c8b9163eff4f8ca15b290be4ad88d7...,female,1960 - 1970,Employed,True,[0-1],flat,heart disease
3,9bf1d2f6590d596ef1ac918c720d28db22632b9ecd78f7...,female,1940 - 1950,Retired,True,[0],room,endometriosis
4,4833d00b5f560752215b61c2115a4835c34afc1354e6a5...,female,1940 - 1950,Retired,True,[0],flat,gastritis
...,...,...,...,...,...,...,...,...
1995,076e99bfa49dc9d10347b2daf236a928d5d5100c44833a...,male,1990 - 2000,Student,False,[1+],house,gastritis
1996,5efd9f9dfc960d509c59a666b709e2f706705e2f8ab4c3...,male,1990 - 2000,Student,False,[0-1],house,gastritis
1997,f23ff2be0dbfeba2a08ed93084bc41dc4f582edfe9f6c7...,female,1940 - 1950,Retired,True,[0],house,skin cancer
1998,3c86137916913fc99989fa2b7bb55bd4b119bb076df633...,male,1970 - 1980,Employed,True,[0],flat,diabetes


In [148]:
final_df.nunique()

id               1914
gender              2
dob                 8
employment          4
children            2
commute_time        3
accommodation       3
disease            13
dtype: int64

In [149]:
df

,id,gender,dob,state,employment,children,commute_time,accommodation,disease
0,9b458418f849bb3e3b4fddac0f74488a568abba21aa0de...,female,1950 - 1960,Unknown,Retired,True,[0],house,Alzheimer's disease
1,4b91f8e668c232635e7637e3ce21b1acf04291cddf3f77...,female,1940 - 1950,(TX) Texas,Retired,True,[0],flat,multiple sclerosis
2,cb079ec3ffa5cce8c8b9163eff4f8ca15b290be4ad88d7...,female,1960 - 1970,(IA) Iowa,Employed,True,[0-1],flat,heart disease
3,9bf1d2f6590d596ef1ac918c720d28db22632b9ecd78f7...,female,1940 - 1950,(WI) Wisconsin,Retired,True,[0],room,endometriosis
4,4833d00b5f560752215b61c2115a4835c34afc1354e6a5...,female,1940 - 1950,(NE) Nebraska,Retired,True,[0],flat,gastritis
...,...,...,...,...,...,...,...,...,...
1995,076e99bfa49dc9d10347b2daf236a928d5d5100c44833a...,male,1990 - 2000,(LA) Louisiana,Student,False,[1+],house,gastritis
1996,5efd9f9dfc960d509c59a666b709e2f706705e2f8ab4c3...,male,1990 - 2000,(MI) Michigan,Student,False,[0-1],house,gastritis
1997,f23ff2be0dbfeba2a08ed93084bc41dc4f582edfe9f6c7...,female,1940 - 1950,(CA) California,Retired,True,[0],house,skin cancer
1998,3c86137916913fc99989fa2b7bb55bd4b119bb076df633...,male,1970 - 1980,(KY) Kentucky,Employed,True,[0],flat,diabetes
